# Bond Machine Learning Portfolio Example

This notebook implements machine learning portfolios for corporate bonds, similar to the equity ML portfolio example but adapted for fixed income.

You can run each cell individually and review the outputs in chunks.


In [ ]:
# Imports and setup
import pandas as pd
import polars as pl
import numpy as np
import wrds
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import datetime as dt
import time
import warnings
warnings.filterwarnings('ignore')


## Parameters
Set the training and testing periods, refit frequency, and number of portfolios.


In [ ]:
# Set parameters
training_start_year = 1980  # Start of training data
training_end_year = 1999    # End of initial training period
test_start_year = 2000      # Start of testing/prediction period
refit_period = 2            # Refit model every 2 years
n_portfolios = 5            # Number of portfolios to form
print(f'Training period: {training_start_year}-{training_end_year}')
print(f'Testing period: {test_start_year}-present')
print(f'Refitting every {refit_period} years')


## Step 1: Get bond returns from WRDS


In [ ]:
print('Connecting to WRDS...')
wrds_conn = wrds.Connection()

print('\nRetrieving bond returns from WRDS...')
start_time = time.time()
bond_returns = wrds_conn.raw_sql("""
    SELECT cusip_id, date, ret_eom as ret, price_eom as price,
           t_volume, t_spread, t_yld_pt
    FROM wrds_bond.bondret
    WHERE date >= '1980-01-01'
    AND ret_eom IS NOT NULL
""", date_cols=['date'])
print(f'Retrieved {len(bond_returns)} bond return observations')
print(f'Time elapsed: {time.time() - start_time:.2f} seconds')


## Step 2: Get bond characteristics from FISD


In [ ]:
print('\nRetrieving bond characteristics from WRDS...')
start_time = time.time()
bond_chars = wrds_conn.raw_sql("""
    SELECT cusip_id, offering_date, maturity, 
           coupon, offering_amt, 
           security_level, bond_type, 
           convertible, callable, putable,
           rule_144a, rating_class
    FROM fisd.fisd_mergedissue
""", date_cols=['offering_date', 'maturity'])
print(f'Retrieved characteristics for {len(bond_chars)} bonds')
print(f'Time elapsed: {time.time() - start_time:.2f} seconds')


## Step 3: Get the bond-equity linking table


In [ ]:
print('\nRetrieving bond-equity linking table...')
start_time = time.time()
bond_equity_link = wrds_conn.raw_sql("""
    SELECT cusip, permno, link_startdt as start_date, 
           link_enddt as end_date
    FROM wrds_bond.bondcrsp_link
""", date_cols=['start_date', 'end_date'])
print(f'Retrieved {len(bond_equity_link)} bond-equity links')
print(f'Time elapsed: {time.time() - start_time:.2f} seconds')


## Step 4: Convert to Polars and calculate bond features


In [ ]:
print('\nConverting data to polars format...')
bond_returns = pl.from_pandas(bond_returns)
bond_chars = pl.from_pandas(bond_chars)
bond_equity_link = pl.from_pandas(bond_equity_link)

print('\nCalculating bond features...')
rating_map = {
        'AAA': 10, 'AA+': 9.7, 'AA': 9.3, 'AA-': 9, 
        'A+': 8.7, 'A': 8.3, 'A-': 8, 
        'BBB+': 7.7, 'BBB': 7.3, 'BBB-': 7, 
        'BB+': 6.7, 'BB': 6.3, 'BB-': 6, 
        'B+': 5.7, 'B': 5.3, 'B-': 5, 
        'CCC+': 4.7, 'CCC': 4.3, 'CCC-': 4, 
        'CC': 3, 'C': 2, 'D': 1
    }
bond_data = bond_returns.join(
        bond_chars, 
        left_on='cusip_id', 
        right_on='cusip_id',
        how='left'
    )
bond_data = bond_data.with_columns([
        (pl.col('maturity').dt.epoch_days() - pl.col('date').dt.epoch_days()).div(365).alias('time_to_maturity'),
        (pl.col('date').dt.epoch_days() - pl.col('offering_date').dt.epoch_days()).div(365).alias('bond_age'),
        pl.col('convertible').cast(pl.Int32).alias('convertible_dummy')
        # Add more feature engineering as in your script
    ])


## Step 5: Merge with equity predictors (optional, if available)

If you have equity predictors from OpenAP, merge here. Otherwise, continue with bond data only.


In [ ]:
import openassetpricing as oap
openap = oap.OpenAP()
nsignals_for_ml = 20  # Adjust for your memory
try:
    print('Downloading equity predictors...')
    equity_predictors = openap.dl_all_signals('polars')
    # Merge/join as in your script
except Exception as e:
    print(f'Error downloading equity predictors: {e}')
    print('Continuing with bond data only...')


## Step 6: Prepare data for machine learning


In [ ]:
# Continue with your ML preparation, filling missing values, etc.
# ...


## Step 7: Model Training (Neural Network and OLS)


In [ ]:
# Train models as in your script
# ...


## Step 8: Form Portfolios and Analyze Performance


In [ ]:
# Portfolio formation and performance analysis
# ...


## Step 9: Plotting and Saving Results


In [ ]:
# Plotting and saving as in your script
# ...


## Summary
Bond ML Portfolio analysis complete!
